In [99]:
import re
import pandas as pd
import numpy as np
import funcs 
import importlib
import unicodecsv as csv
import pickle
import random
import gensim
from gensim import corpora, models
from timeit import default_timer as timer
from datetime import datetime, timedelta
import nltk
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import spacy
from collections import Counter

importlib.reload(funcs)

<module 'funcs' from 'D:\\_projekty\\praca\\funcs.py'>

## Załadowanie danych oczyszczonych i zlematyzowanych oraz dalsze czyszczenie

In [2]:
sp = pd.read_csv('00_dane/rawest.csv',  sep='|', encoding='UTF-8')

In [19]:
dane = sp[~sp['speech_rawest'].isnull()]

In [21]:
%%time
dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: re.sub('[\W\d]',' ', x.lower()))

#czas dla apply : 19s

Wall time: 14.7 s


C:\Users\Magda\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
nlp = spacy.load('pl_model')
stop = set(nlp.Defaults.stop_words)
stop.add('zł')
stop.add('pkt')
stop.add('art')
stop.add('ustawy')
stop.add('r')

In [24]:
%%time
dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: [word for word in x.split() if word not in stop])

Wall time: 7.59 s


C:\Users\Magda\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Zapisanie pliku w takiej postaci

In [27]:
dane.to_csv('00_dane/dane_TM.csv', sep='|', encoding='UTF-8', index=False)


## Podzbiór testowy (analiza za: https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24)

In [55]:
test = dane['speech_rawest'][:50000]

In [44]:
print(test[113])

['rok', 'trwać', 'praca', 'legislacyjny', 'związany', 'opracowanie', 'nowy', 'ustawa', 'ochrona', 'grunt', 'rolny', 'leśny', 'dzień', 'grudzień', 'rok', 'ustawa', 'zostać', 'uchwalić', 'sejm', 'rzeczypospolity', 'polski', 'zasadniczy', 'sprawa', 'duży', 'osiągnięcie', 'omawiać', 'ustawa', 'decentralizacja', 'decyzja', 'dotyczyć', 'ochrona', 'grunt', 'wykorzystać', 'zamierzyć', 'cel', 'ustawa', 'zostać', 'osiągnąć', 'wyeliminować', 'niekorzystny', 'rolnictwo', 'produkcja', 'rolny', 'uregulowanie', 'prawny', 'dostosować', 'przepis', 'ochrona', 'grunt', 'rolny', 'leśny', 'zmienić', 'uwarunkowanie', 'społeczny', 'gospodarczy', 'ustawa', 'poszerzać', 'swoboda', 'właściciel', 'grunt', 'ograniczać', 'możliwość', 'wywłaszczenie', 'zwiększać', 'samodzielność', 'władza', 'lokalny', 'zmierzać', 'ochrona', 'wartościowy', 'grunt', 'rolny', 'leśny', 'zawierać', 'szereg', 'nowy', 'istotny', 'zdanie', 'klub', 'parlamentarny', 'sojusz', 'lewica', 'demokratyczny', 'rozstrzygnięcie', 'obejmować', 'ustawa

In [56]:
dictionary = gensim.corpora.Dictionary(test)

In [32]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 agent
1 amnestyjny
2 ażeby
3 bezprecensnsowy
4 bezzasadny
5 budżetowy
6 celowo
7 chcieć
8 cierpieć
9 ciągle
10 contry


Usuwanie słów rzadkich i zbyt częstych i ograniczeni liczby tokenów

In [57]:
dictionary.filter_extremes(no_above=0.4, no_below=15 )

In [58]:
len(dictionary)

18888

In [40]:
print(dictionary)

Dictionary(8912 unique tokens: ['agent', 'ażeby', 'bezprecensnsowy', 'bezzasadny', 'budżetowy']...)


## Bag of Words
Dla każdego dokumentu tworzymy słownik zawierający informację jak dużo słów się pojawia i jak często.

In [59]:
BOW = [dictionary.doc2bow(speech) for speech in test]

In [48]:
BOW_113 = BOW[113]
for i in range(len(BOW_113)):
    print("Word {} (\"{}\") appears {} time.".format(BOW_113[i][0], 
                                               dictionary[BOW_113[i][0]], 
BOW_113[i][1]))

Word 14 ("decyzja") appears 3 time.
Word 18 ("dotyczyć") appears 3 time.
Word 20 ("duży") appears 1 time.
Word 35 ("istotny") appears 1 time.
Word 42 ("komisja") appears 2 time.
Word 65 ("niebezpieczny") appears 1 time.
Word 114 ("polski") appears 1 time.
Word 115 ("poprawka") appears 6 time.
Word 117 ("poprzez") appears 2 time.
Word 129 ("praca") appears 1 time.
Word 144 ("przyjąć") appears 2 time.
Word 159 ("sejm") appears 2 time.
Word 167 ("sprawa") appears 2 time.
Word 173 ("stanowisko") appears 3 time.
Word 211 ("wprowadzić") appears 1 time.
Word 215 ("wszystek") appears 1 time.
Word 216 ("wydawać") appears 1 time.
Word 230 ("zakres") appears 1 time.
Word 240 ("zdanie") appears 1 time.
Word 246 ("zostać") appears 2 time.
Word 268 ("demokratyczny") appears 3 time.
Word 292 ("klub") appears 3 time.
Word 297 ("lewica") appears 3 time.
Word 316 ("nowy") appears 2 time.
Word 324 ("odnosić") appears 1 time.
Word 327 ("określić") appears 2 time.
Word 329 ("parlamentarny") appears 2 time.

## TF-IDF
Oblicznie wartości tf-idf

In [60]:
tfidf = models.TfidfModel(BOW)
corpus_tfidf = tfidf[BOW]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.05282992042408906),
 (1, 0.08023904348830177),
 (2, 0.06567202781548985),
 (3, 0.0577810555986525),
 (4, 0.04991313038230942),
 (5, 0.05359437398375828),
 (6, 0.05100272276250704),
 (7, 0.034757299893813265),
 (8, 0.06102645254958782),
 (9, 0.07088930513846323),
 (10, 0.025120994845624463),
 (11, 0.12374710379250481),
 (12, 0.10384569089277046),
 (13, 0.03791792995869988),
 (14, 0.03493319055639545),
 (15, 0.06192046880189102),
 (16, 0.03410147037483845),
 (17, 0.03637815507963329),
 (18, 0.04452952420732084),
 (19, 0.0125191489463708),
 (20, 0.07164071674120644),
 (21, 0.029873369641820035),
 (22, 0.06118301440869654),
 (23, 0.05109120809980182),
 (24, 0.05348162349451137),
 (25, 0.03403925900734858),
 (26, 0.024219307979655946),
 (27, 0.020487971199778892),
 (28, 0.019006785777984175),
 (29, 0.029757475168270844),
 (30, 0.06262357928279715),
 (31, 0.055126303002906094),
 (32, 0.03103857407420638),
 (33, 0.13451308012933105),
 (34, 0.01917669019515162),
 (35, 0.0189325971630841

## LDA oparte o BOW

In [63]:
lda_model = gensim.models.LdaMulticore(BOW, id2word=dictionary,num_topics=10, passes=2, workers=2)

In [64]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.014*"zmiana" + 0.012*"poprawka" + 0.010*"przepis" + 0.007*"klub" + 0.006*"arta" + 0.006*"dzień" + 0.006*"zakres" + 0.006*"usta" + 0.005*"proponować" + 0.005*"przyjąć"
Topic: 1 
Words: 0.008*"spółka" + 0.007*"przedsiębiorstwo" + 0.007*"gospodarczy" + 0.007*"bank" + 0.006*"skarb" + 0.006*"prywatyzacja" + 0.006*"gospodarka" + 0.005*"rynek" + 0.005*"firma" + 0.004*"przedsiębiorca"
Topic: 2 
Words: 0.006*"narodowy" + 0.006*"polska" + 0.005*"wysoki" + 0.005*"wojskowy" + 0.005*"swój" + 0.004*"dzień" + 0.004*"żołnierz" + 0.004*"służba" + 0.004*"człowiek" + 0.004*"uczelnia"
Topic: 3 
Words: 0.024*"dziecko" + 0.014*"osoba" + 0.014*"rodzina" + 0.007*"praca" + 0.006*"rzecznik" + 0.006*"społeczny" + 0.006*"pomoc" + 0.005*"kobieta" + 0.005*"świadczenie" + 0.005*"niepełnosprawny"
Topic: 4 
Words: 0.013*"rząd" + 0.009*"problem" + 0.008*"praca" + 0.007*"sejm" + 0.007*"pewien" + 0.006*"budżet" + 0.006*"powiedzieć" + 0.005*"wniosek" + 0.005*"społeczny" + 0.005*"chodzić"
Topic: 5 
Words

## LDA oparte o TF-IDF

In [66]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.004*"złoty" + 0.004*"budżet" + 0.003*"podatek" + 0.003*"mld" + 0.003*"rząd" + 0.003*"bank" + 0.003*"wzrost" + 0.002*"budżetowy" + 0.002*"pieniądz" + 0.002*"wydatek"
Topic: 1 Word: 0.005*"rolnik" + 0.004*"rolnictwo" + 0.004*"rolny" + 0.004*"europejski" + 0.003*"unia" + 0.003*"poprawka" + 0.002*"rynek" + 0.002*"rozwój" + 0.002*"podatek" + 0.002*"gospodarstwo"
Topic: 2 Word: 0.002*"rząd" + 0.002*"polska" + 0.002*"premier" + 0.002*"marszałek" + 0.002*"człowiek" + 0.002*"sejm" + 0.002*"prosić" + 0.002*"polityczny" + 0.001*"powiedzieć" + 0.001*"polak"
Topic: 3 Word: 0.003*"dziecko" + 0.003*"praca" + 0.003*"osoba" + 0.002*"zdrowie" + 0.002*"zmiana" + 0.002*"poprawka" + 0.002*"służba" + 0.002*"sąd" + 0.002*"świadczenie" + 0.002*"rodzina"
Topic: 4 Word: 0.011*"poprawka" + 0.006*"senat" + 0.003*"klub" + 0.003*"zmiana" + 0.003*"przyjąć" + 0.002*"głosować" + 0.002*"sprawozdanie" + 0.002*"nieruchomość" + 0.002*"zgłosić" + 0.002*"konwencja"


In [69]:
lda_model_tfidf.print_topics(0)

[(0,
  '0.004*"złoty" + 0.004*"budżet" + 0.003*"podatek" + 0.003*"mld" + 0.003*"rząd" + 0.003*"bank" + 0.003*"wzrost" + 0.002*"budżetowy" + 0.002*"pieniądz" + 0.002*"wydatek"'),
 (3,
  '0.003*"dziecko" + 0.003*"praca" + 0.003*"osoba" + 0.002*"zdrowie" + 0.002*"zmiana" + 0.002*"poprawka" + 0.002*"służba" + 0.002*"sąd" + 0.002*"świadczenie" + 0.002*"rodzina"'),
 (4,
  '0.011*"poprawka" + 0.006*"senat" + 0.003*"klub" + 0.003*"zmiana" + 0.003*"przyjąć" + 0.002*"głosować" + 0.002*"sprawozdanie" + 0.002*"nieruchomość" + 0.002*"zgłosić" + 0.002*"konwencja"'),
 (2,
  '0.002*"rząd" + 0.002*"polska" + 0.002*"premier" + 0.002*"marszałek" + 0.002*"człowiek" + 0.002*"sejm" + 0.002*"prosić" + 0.002*"polityczny" + 0.001*"powiedzieć" + 0.001*"polak"'),
 (1,
  '0.005*"rolnik" + 0.004*"rolnictwo" + 0.004*"rolny" + 0.004*"europejski" + 0.003*"unia" + 0.003*"poprawka" + 0.002*"rynek" + 0.002*"rozwój" + 0.002*"podatek" + 0.002*"gospodarstwo"')]

In [100]:
funcs.print_topics(lda_model_tfidf.print_topics(-1))

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4
0,złoty,rolnik,rząd,dziecko,poprawka
1,budżet,rolnictwo,polska,praca,senat
2,podatek,rolny,premier,osoba,klub
3,mld,europejski,marszałek,zdrowie,zmiana
4,rząd,unia,człowiek,zmiana,przyjąć
5,bank,poprawka,sejm,poprawka,głosować
6,wzrost,rynek,prosić,służba,sprawozdanie
7,budżetowy,rozwój,polityczny,sąd,nieruchomość
8,pieniądz,podatek,powiedzieć,świadczenie,zgłosić
9,wydatek,gospodarstwo,polak,rodzina,konwencja


In [85]:
lda_model_tfidf.print_topics(-1)[0]

(0,
 '0.004*"złoty" + 0.004*"budżet" + 0.003*"podatek" + 0.003*"mld" + 0.003*"rząd" + 0.003*"bank" + 0.003*"wzrost" + 0.002*"budżetowy" + 0.002*"pieniądz" + 0.002*"wydatek"')

In [89]:
pd.Series(a[0][1].split(' + ')).apply(lambda x : re.sub(r'[\d\.\*\"\']', '', x))

0        poprawka
1           senat
2            klub
3          zmiana
4         przyjąć
5        głosować
6    sprawozdanie
7    nieruchomość
8         zgłosić
9       konwencja
Name: Topic 4, dtype: object

In [91]:
topic_df = pd.DataFrame()
for topic in lda_model_tfidf.print_topics(-1):
    topic_df['Topic ' + str(topic[0])] = pd.Series(topic[1].split(' + ')).apply(lambda x : re.sub(r'[\d\.\*\"\']', '', x))

In [93]:
topic_df

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4
0,złoty,rolnik,rząd,dziecko,poprawka
1,budżet,rolnictwo,polska,praca,senat
2,podatek,rolny,premier,osoba,klub
3,mld,europejski,marszałek,zdrowie,zmiana
4,rząd,unia,człowiek,zmiana,przyjąć
5,bank,poprawka,sejm,poprawka,głosować
6,wzrost,rynek,prosić,służba,sprawozdanie
7,budżetowy,rozwój,polityczny,sąd,nieruchomość
8,pieniądz,podatek,powiedzieć,świadczenie,zgłosić
9,wydatek,gospodarstwo,polak,rodzina,konwencja
